In [2]:
import numpy as np
import cv2
cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    face_crop = []
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,255,0), 2)
        face_crop.append(gray[y:y+h, x:x+w])
        
    cv2.imshow('img', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


-1

Notes: There seems to be a bug with OpenCV running in Jupyter Notebook, cv2.imshow() causes the program to hang. The last line 'cv2.waitKey(1)' somehow bypasses it and allows the cv2.imshow() to work. Even so, you still need to restart the kernel after stopping the run.

In [4]:
# reboot kernel
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}